In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import uuid

In [2]:
# !pip install surt
# !pip install validators

In [3]:
# Two handy libraries for checking if a URL is valid or not
# import surt
import validators

## SEC-Registered

In [27]:
df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_02_15_2023.csv')

C:\Users\14097\AppData\Local\Temp\ipykernel_15316\3704932919.py:1: DtypeWarning: Columns (49,217,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sec = pd.read_csv('data/sec/IA_FIRM_SEC_Feed_02_15_2023.csv')


In [28]:
# Remove long prefix at the beginning of many column names
df_sec.columns = df_sec.columns.str.removeprefix('FormInfo.Part1A.Item')

In [49]:
df_sec['uuid'] = df_sec.apply(lambda _: uuid.uuid4(), axis=1)

In [50]:
df_sec['uuid']

0        04dfae9a-53e9-484b-bcdf-d0953fed1c7b
1        230a8c84-50e2-4083-ae1e-5395dd480478
2        526a51a5-542a-4f0a-958a-7d60dadbde4f
3        ec1a2ff0-a2dc-418b-82c3-42478ee8efd3
4        448d5008-75f1-462d-b1b3-18a38f78e2fc
                         ...                 
21070    337d3d51-0676-4bbf-a66d-7c393654f1be
21071    b23c907d-f679-4f4a-b651-77b073fe0b07
21072    fbbe881c-a89c-46f5-9a34-84b71a95b832
21073    151fa1c2-6424-4817-83e2-a402501f565e
21074    039c1f54-0ec2-4bee-920c-de6150ede399
Name: uuid, Length: 21075, dtype: object

In [29]:
sorted(df_sec.columns.to_list())

['1.@Q1F5',
 '1.@Q1I',
 '1.@Q1M',
 '1.@Q1N',
 '1.@Q1O',
 '1.@Q1ODesc',
 '1.@Q1P',
 '1.WebAddrs',
 '1.WebAddrs.WebAddr',
 '10A.@Q10A',
 '11.@Q11',
 '11A.@Q11A1',
 '11A.@Q11A2',
 '11B.@Q11B1',
 '11B.@Q11B2',
 '11C.@Q11C1',
 '11C.@Q11C2',
 '11C.@Q11C3',
 '11C.@Q11C4',
 '11C.@Q11C5',
 '11D.@Q11D1',
 '11D.@Q11D2',
 '11D.@Q11D3',
 '11D.@Q11D4',
 '11D.@Q11D5',
 '11E.@Q11E1',
 '11E.@Q11E2',
 '11E.@Q11E3',
 '11E.@Q11E4',
 '11F.@Q11F',
 '11G.@Q11G',
 '11H.@Q11H1A',
 '11H.@Q11H1B',
 '11H.@Q11H1C',
 '11H.@Q11H2',
 '2A.@Q2A1',
 '2A.@Q2A10',
 '2A.@Q2A11',
 '2A.@Q2A12',
 '2A.@Q2A13',
 '2A.@Q2A2',
 '2A.@Q2A4',
 '2A.@Q2A5',
 '2A.@Q2A6',
 '2A.@Q2A7',
 '2A.@Q2A8',
 '2A.@Q2A9',
 '2B.@Q2B1',
 '2B.@Q2B2',
 '2B.@Q2B3',
 '3A.@OrgFormNm',
 '3A.@OrgFormOthNm',
 '3B.@Q3B',
 '3C.@CntryNm',
 '3C.@StateCD',
 '5A.@TtlEmp',
 '5B.@Q5B1',
 '5B.@Q5B2',
 '5B.@Q5B3',
 '5B.@Q5B4',
 '5B.@Q5B5',
 '5B.@Q5B6',
 '5C.@Q5C1',
 '5C.@Q5C1Oth',
 '5C.@Q5C2',
 '5D.@Q5D1A',
 '5D.@Q5D1B',
 '5D.@Q5D1C',
 '5D.@Q5D1D',
 '5D.@Q5D1E',
 '5D.@

Number of Advisers -- everything

In [11]:
df_sec.shape[0]

21075

In [26]:
df_sec.shape

(21075, 280)

In [42]:
df_sec['Info.@SECNb'].unique().size

21074

In [32]:
df_secno = df_sec['Info.@SECNb']

In [33]:
df_secno

0         801-73527
1        801-118322
2        801-106933
3        801-108550
4         801-18270
            ...    
21070    802-127315
21071    802-127286
21072    801-127223
21073    802-127319
21074    802-127285
Name: Info.@SECNb, Length: 21075, dtype: object

In [34]:
df_secnb = df_sec['Info.@FirmCrdNb']

In [61]:
df_secnb.shape[0]

21075

In [12]:
df_sec['large_or_mid_sized'] = np.where((df_sec['FormInfo.Part1A.Item2A.@Q2A1'] == 'Y') | (df_sec['FormInfo.Part1A.Item2A.@Q2A2'] == 'Y'),
                                        True,
                                        False)

In [13]:
df_sec['robo'] = np.where(df_sec['FormInfo.Part1A.Item2A.@Q2A11'] == 'Y',
                          True, False)

In [14]:
(df_sec.loc[(df_sec['FormInfo.Part1A.Item2A.@Q2A1'] == 'Y') | (df_sec['FormInfo.Part1A.Item2A.@Q2A2'] == 'Y'),:].index == df_sec[df_sec['large_or_mid_sized']].index).all()

True

Number of Advisers -- just the "regular" firms

In [15]:
df_sec_reg = df_sec[df_sec['large_or_mid_sized']].copy()
df_sec_reg.shape[0]

13794

Number of Robo Advisers

In [16]:
df_sec_robo = df_sec[df_sec['robo']].copy()
df_sec_robo.shape[0]

269

## State-Registered Advisers 

In [17]:
df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_02_15_2023.csv')

C:\Users\14097\AppData\Local\Temp\ipykernel_16888\3518042785.py:1: DtypeWarning: Columns (35,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,270,277,299,306,309,310,313) have mixed types. Specify dtype option on import or set low_memory=False.
  df_state = pd.read_csv('data/sec/IA_FIRM_STATE_Feed_02_15_2023.csv')


In [18]:
sorted(df_state.columns.to_list())

['ERA.Rgltrs',
 'ERA.Rgltrs.Rgltr',
 'ERA.Rgltrs.Rgltr.@Cd',
 'ERA.Rgltrs.Rgltr.@Dt',
 'ERA.Rgltrs.Rgltr.@St',
 'Filing.@Dt',
 'Filing.@FormVrsn',
 'FormInfo.Part1A.Item1.@Q1F5',
 'FormInfo.Part1A.Item1.@Q1I',
 'FormInfo.Part1A.Item1.@Q1M',
 'FormInfo.Part1A.Item1.@Q1N',
 'FormInfo.Part1A.Item1.@Q1O',
 'FormInfo.Part1A.Item1.@Q1ODESC',
 'FormInfo.Part1A.Item1.@Q1P',
 'FormInfo.Part1A.Item1.WebAddrs',
 'FormInfo.Part1A.Item1.WebAddrs.WebAddr',
 'FormInfo.Part1A.Item10A.@Q10A',
 'FormInfo.Part1A.Item11.@Q11',
 'FormInfo.Part1A.Item11A.@Q11A1',
 'FormInfo.Part1A.Item11A.@Q11A2',
 'FormInfo.Part1A.Item11B.@Q11B1',
 'FormInfo.Part1A.Item11B.@Q11B2',
 'FormInfo.Part1A.Item11C.@Q11C1',
 'FormInfo.Part1A.Item11C.@Q11C2',
 'FormInfo.Part1A.Item11C.@Q11C3',
 'FormInfo.Part1A.Item11C.@Q11C4',
 'FormInfo.Part1A.Item11C.@Q11C5',
 'FormInfo.Part1A.Item11D.@Q11D1',
 'FormInfo.Part1A.Item11D.@Q11D2',
 'FormInfo.Part1A.Item11D.@Q11D3',
 'FormInfo.Part1A.Item11D.@Q11D4',
 'FormInfo.Part1A.Item11D.@Q11D5

In [51]:
df_state['uuid'] = df_state.apply(lambda _: uuid.uuid4(), axis=1)

In [54]:
df_state['uuid']

0        12a956dd-13b4-42d2-8ee2-362a7f870512
1        4c221b39-b5eb-4a59-afa5-3b00513ad73d
2        aa4af946-29d9-4b0d-b18b-41e8246694c6
3        026e07ef-7160-4ae1-902f-bd6623e411b5
4        edfe0f5c-0f9c-4b5b-ab96-ca8bf6226d7c
                         ...                 
21139    4fac669a-8536-413d-bb48-3efe1c78aa23
21140    856e6a55-5b24-4c0e-9744-2223dd75a92d
21141    34bad470-3090-4252-82f4-fddb09079a3d
21142    0be2fe1b-7993-4af9-ac6a-9e73befb4f62
21143    fd79f81a-a4d6-428f-8bf6-8aeb7ba7bec9
Name: uuid, Length: 21144, dtype: object

Some Advisers are "Exempt Reporting Advisers", they are typically VCs 
https://learn.angellist.com/articles/exempt-reporting-advisor

In [19]:
df_state_exempt = df_state[(df_state['ERA.Rgltrs'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Cd'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@Dt'].notnull() |
                            df_state['ERA.Rgltrs.Rgltr.@St'].notnull() )].copy()

In [21]:
df_state_reg = df_state[(df_state['ERA.Rgltrs'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Cd'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@Dt'].isnull() &
                        df_state['ERA.Rgltrs.Rgltr.@St'].isnull() )].copy()

In [22]:
df_state_reg.shape[0] + df_state_exempt.shape[0] == df_state.shape[0]

True

In [25]:
df_state.shape[0]

21144

In [27]:
df_state.shape

(21144, 314)

In [43]:
df_state['Info.@SECNb'].unique().size

3821

In [44]:
df_state['Info.@FirmCrdNb'].unique().size

21144

In [36]:
df_statecrb = df_state['Info.@FirmCrdNb']

In [37]:
df_statecrb

0        146099
1        289221
2         23940
3        155220
4        171863
          ...  
21139    324692
21140    324710
21141    324715
21142    324698
21143    324726
Name: Info.@FirmCrdNb, Length: 21144, dtype: int64

# Combine

In [70]:
df_all = pd.merge(df_sec, df_state, on=['Info.@FirmCrdNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [74]:
df_all.shape

(1759, 591)

In [73]:
df_all['Info.@FirmCrdNb'].unique().size

1759

In [78]:
df_all['Info.@SECNb_x'].unique().size

1759

In [79]:
df_all.sample(3)

,Info.@SECRgnCD,Info.@FirmCrdNb,Info.@SECNb_x,Info.@BusNm,Info.@LegalNm,Info.@UmbrRgstn_x,MainAddr.@Strt1_x,MainAddr.@Strt2_x,MainAddr.@City_x,MainAddr.@State_x,...,FormInfo.Part1B.ItemJ.@Q1BJ2BNone,FormInfo.Part1A.Item9E.@Q9E_y,ERA.Rgltrs.Rgltr,FormInfo.Part1B.ItemH.@Q1B2HNScrtsNvsmtAm,FormInfo.Part1A.Item1.@Q1ODESC,FormInfo.Part1A.Item5D.@Q5D1MOth,FormInfo.Part1A.Item5H.@Q5HMT500_y,FormInfo.Part1A.Item5C.@Q5C1Oth_y,FormInfo.Part1A.Item5D.@Q5D2MOth,uuid_y
693,SFRO,163037,802-75543,"MERUS CAPITAL MANAGEMENT COMPANY, LLC","MERUS CAPITAL MANAGEMENT COMPANY, LLC",NaN,2180 SAND HILL ROAD,SUITE 220,MENLO PARK,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23319e33-b79a-415a-86e4-be58065dccf8
443,SFRO,173304,802-106586,"TOLA CAPITAL MANAGEMENT, LLC","TOLA CAPITAL MANAGEMENT, LLC",NaN,520 E DENNY WAY,NaN,SEATTLE,WA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34ebd575-21b5-4d01-b802-76a2ad9f454e
1185,NYRO,313822,802-121145,"ATYPICAL VENTURES GP I, L.L.C.","ATYPICAL VENTURES GP I, L.L.C.",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cf31fdb3-c236-4344-a405-6da97c2ad517


# Get All Web Addresses

In [23]:
def get_all_urls(df, valid_only=True):
    df_urls = df.loc[df['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].notnull(),
                    ['Info.@FirmCrdNb',
                     'FormInfo.Part1A.Item1.WebAddrs.WebAddr']
                    ]
    df_urls.rename(columns={'FormInfo.Part1A.Item1.WebAddrs.WebAddr':'url'}, inplace=True)
    df_urls['url'] = df_urls['url'].apply(lambda x: eval(x.lower()) if ('[' in x and ']' in x) else x.lower())
    df_urls = df_urls.explode('url').reset_index(drop=True)
    df_urls['url_valid'] = df_urls['url'].apply(validators.url)
    if valid_only:
        df_urls = df_urls.loc[df_urls['url_valid'] == True,:]
    return df_urls

In [24]:
df_sec_reg_urls = get_all_urls(df_sec_reg, valid_only=False)